<a href="https://colab.research.google.com/github/junior825/BERT/blob/main/Natural_Language_Processing_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"parkyongchan","key":"3a4d692af68deb0ef0e7c9f388541a58"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         90           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes         94           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        956           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        160           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1760           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      14400           False  
house-pr

In [4]:
!kaggle competitions download -c nlp-getting-started

!kaggle competitions files -c nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 31.2MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 57.1MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 19.6MB/s]
name                    size  creationDate         
---------------------  -----  -------------------  
train.csv              965KB  2019-12-16 20:36:20  
sample_submission.csv   22KB  2019-12-16 20:36:20  
test.csv               411KB  2019-12-16 20:36:20  


In [5]:
import pandas as pd

test = pd.DataFrame(pd.read_csv('./test.csv')) # test feature
submission = pd.DataFrame(pd.read_csv('./sample_submission.csv')) # Prediction
train = pd.DataFrame(pd.read_csv('./train.csv')) # train_tata => feature / target

train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
print(train.isnull().sum())
print(len(train))
print(train.columns)

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64
7613
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


In [7]:
train_data = train.drop(['id','keyword','location'], axis=1)

train_data

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [8]:
print(train_data.groupby('target').size())

target
0    4342
1    3271
dtype: int64


In [54]:
import re

print(train_data['text'])
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!·]', ' ', inputString) 
  text_rmv = ' '.join(text_rmv.split())
  return text_rmv

train_text_feature = [clean_text(str(s)) for s in train_data['text']]
train_text_feature

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object


['Our Deeds are the Reason of this earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask Canada',
 "All residents asked to 'shelter in place' are being notified by officers No other evacuation or shelter in place orders are expected",
 '13 000 people receive wildfires evacuation orders in California',
 'Just got sent this photo from Ruby Alaska as smoke from wildfires pours into a school',
 'RockyFire Update > California Hwy 20 closed in both directions due to Lake County fire CAfire wildfires',
 'flood disaster Heavy rain causes flash flooding of streets in Manitou Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area",
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded hah WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK flooding',
 "rain

In [55]:
from sklearn.model_selection import train_test_split

train_feature, val_feature, train_target, val_target = train_test_split(train_text_feature, train_data['target'], train_size = 0.8, random_state = 47)

print(train_feature[0])

Cuban leader extends sympathy to Vietnam over flooding at http t co xLq8G6vb2r


In [56]:
!pip install transformers
from transformers import BertTokenizerFast, TFBertModel

BERT_MODEL = 'bert-base-uncased'

tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL)

train_tokenizer = tokenizer(train_feature, return_tensors='tf', truncation = True, padding = True)
val_tokenizer = tokenizer(val_feature, return_tensors='tf', truncation = True, padding = True)


In [57]:
print(train_tokenizer[0].tokens)

['[CLS]', 'cuban', 'leader', 'extends', 'sympathy', 'to', 'vietnam', 'over', 'flooding', 'at', 'http', 't', 'co', 'xl', '##q', '##8', '##g', '##6', '##vb', '##2', '##r', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [58]:
import tensorflow as tf

with tf.device('/device:GPU:0'):
  train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_tokenizer),
    train_target
    ))
  
  val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_tokenizer),
    val_target
    ))

In [62]:
from transformers import BertTokenizer, TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2, from_pt=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

with tf.device('/device:GPU:0'): # device에 있는 GPU:0를 사용하겠다.
  model.fit(
    train_dataset.shuffle(1000).batch(15), 
    epochs = 5, batch_size= 16
    )

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
406/406 [==============================] - 179s 383ms/step - loss: 0.6388 - accuracy: 0.4939
Epoch 2/5
406/406 [==============================] - 156s 383ms/step - loss: 0.7640 - accuracy: 0.5005
Epoch 3/5
406/406 [==============================] - 156s 383ms/step - loss: 0.6240 - accuracy: 0.4744
Epoch 4/5
103/406 [======>.......................] - ETA: 1:56 - loss: 0.5651 - accuracy: 0.4576

KeyboardInterrupt: ignored

In [35]:
import os
MODEL_NAME = 'My-bert-base'
MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

_model/My-bert-base -- Folder create complete 



('_model/My-bert-base/tokenizer_config.json',
 '_model/My-bert-base/special_tokens_map.json',
 '_model/My-bert-base/vocab.txt',
 '_model/My-bert-base/added_tokens.json',
 '_model/My-bert-base/tokenizer.json')

In [37]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True
)

Some layers from the model checkpoint at _model/My-bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at _model/My-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [45]:
test['real'] = submission['target']

test

,text,real
0,Just happened a terrible car crash,0
1,"Heard about #earthquake is different cities, s...",0
2,"there is a forest fire at spot pond, geese are...",0
3,Apocalypse lighting. #Spokane #wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,0
...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0
3259,Storm in RI worse than last hurricane. My city...,0
3260,Green Line derailment in Chicago http://t.co/U...,0
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,0


In [46]:
predicted_label_list = []
predicted_score_list = []

for text in test['text']:
    # predict
    preds_list = text_classifier(text)[0]

    sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
    predicted_label_list.append(sorted_preds_list[0]) # label
    predicted_score_list.append(sorted_preds_list[1]) # score

test['pred'] = predicted_label_list
test['score'] = predicted_score_list
test.head()

#result_data = pd.DataFrame({'text' : test['text'], 'target' : submission['target'], 'predict' : predict})

,text,real,pred,score
0,Just happened a terrible car crash,0,"{'label': 'LABEL_1', 'score': 0.9386705756187439}","{'label': 'LABEL_0', 'score': 0.0613294392824173}"
1,"Heard about #earthquake is different cities, s...",0,"{'label': 'LABEL_1', 'score': 0.9386705756187439}","{'label': 'LABEL_0', 'score': 0.06132937595248..."
2,"there is a forest fire at spot pond, geese are...",0,"{'label': 'LABEL_1', 'score': 0.9386705756187439}","{'label': 'LABEL_0', 'score': 0.0613294392824173}"
3,Apocalypse lighting. #Spokane #wildfires,0,"{'label': 'LABEL_1', 'score': 0.9386705756187439}","{'label': 'LABEL_0', 'score': 0.0613294392824173}"
4,Typhoon Soudelor kills 28 in China and Taiwan,0,"{'label': 'LABEL_1', 'score': 0.9386706948280334}","{'label': 'LABEL_0', 'score': 0.06132932752370..."
